In [ ]:

# from https://github.com/tensorflow/tensorflow/issues/25385


import time
import numpy as np

import tensorflow as tf

layers = tf.keras.layers


In [ ]:
def create_model():
    max_pool = layers.MaxPooling2D((2, 2), (2, 2), padding='same')
    # The model consists of a sequential chain of layers, so tf.keras.Sequential
    # (a subclass of tf.keras.Model) makes for a compact description.
    return tf.keras.Sequential([
        layers.Reshape(
            target_shape=[28, 28, 1],
            input_shape=(28, 28,)),
        layers.Conv2D(2, 5, padding='same', activation=tf.nn.relu),
        max_pool,
        layers.Conv2D(4, 5, padding='same', activation=tf.nn.relu),
        max_pool,
        layers.Flatten(),
        layers.Dense(32, activation=tf.nn.relu),
        layers.Dropout(0.4),
        layers.Dense(10)])


# Define a loss function and accuracy function
def compute_loss(logits, labels):
    return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, logits))


def compute_accuracy(logits, labels):
    return tf.keras.metrics.categorical_accuracy(labels, logits)

# Set up datasets
def mnist_datasets():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    # Numpy defaults to dtype=float64; TF defaults to float32. Stick with float32.
    x_train, x_test = x_train / np.float32(255), x_test / np.float32(255)
    y_train, y_test = y_train.astype(np.int64), y_test.astype(np.int64)
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    return train_dataset, test_dataset



In [ ]:

#@tf.function
#def square_if_positive_vectorized(x):
#  return tf.where(x > 0, x ** 2, x)


#square_if_positive_vectorized(tf.range(-5, 5))

#summary_writer = tf.summary.create_file_writer('/tmp/summaries')
#with summary_writer.as_default():
#  tf.summary.scalar('loss', 0.1, step=42)
  

In [ ]:
@tf.function
def train_step(model, optimizer, images, labels):
    # Record the operations used to compute the loss, so that the gradient
    # of the loss with respect to the variables can be computed.
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        loss = compute_loss(logits, labels)
        accuracy = compute_accuracy(logits, labels)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss, accuracy


In [ ]:
def train(model, optimizer, dataset, log_freq=10):

    """Trains model on `dataset` using `optimizer`."""
    start = time.time()
    # Metrics are stateful. They accumulate values and return a cumulative
    # result when you call .result(). Clear accumulated values with .reset_states()
    avg_loss = tf.keras.metrics.Mean('loss', dtype=tf.float32)
    avg_accuracy = tf.keras.metrics.Mean('accuracy', dtype=tf.float32)
    # Datasets can be iterated over like any other Python iterable.
    for images, labels in dataset:
        loss, accuracy = train_step(model, optimizer, images, labels)
        avg_loss.update_state(loss) #
        avg_loss(loss)
        avg_accuracy(accuracy)
        if tf.equal(optimizer.iterations % log_freq, 0):
            tf.summary.scalar('loss', avg_loss.result(), step=optimizer.iterations)
            tf.summary.scalar('accuracy', avg_accuracy.result(), step=optimizer.iterations)
            avg_loss.reset_states()
            avg_accuracy.reset_states()
            rate = log_freq / (time.time() - start)
            print('Step #%d\tLoss: %.6f (%d steps/sec)' % (optimizer.iterations, loss, rate))
            start = time.time()


def test(model, dataset, step_num):
    """Perform an evaluation of `model` on the examples from `dataset`."""
    avg_loss = tf.keras.metrics.Mean('loss', dtype=tf.float32)
    avg_accuracy = tf.keras.metrics.Mean('accuracy', dtype=tf.float32)

    for (images, labels) in dataset:
        logits = model(images, training=False)
        avg_loss(compute_loss(logits, labels))
        avg_accuracy(compute_accuracy(logits, labels))
    print('Model test set loss: {:0.4f} accuracy: {:0.2f}%'.format(
        avg_loss.result(), avg_accuracy.result() * 100))
    tf.summary.scalar('loss', avg_loss.result(), step=step_num)
    tf.summary.scalar('accuracy', avg_accuracy.result(), step=step_num)


In [ ]:

model = create_model()

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.5)

train_ds, test_ds = mnist_datasets()
train_ds = train_ds.shuffle(60000).batch(100)
test_ds = test_ds.batch(100)

train_summary_writer = tf.summary.create_file_writer('/tmp/summaries/train')
test_summary_writer = tf.summary.create_file_writer('/tmp/summaries/test')

for epoch in range(10):
  start = time.time()
  with train_summary_writer.as_default():
    train(model, optimizer, train_ds)
  end = time.time()
  print('\nTrain time for epoch #{} ({} total steps): {}'.format(epoch + 1, optimizer.iterations, end - start))
  with test_summary_writer.as_default():
    test(model, test_ds, optimizer.iterations)